## Build Functions for ELT

In [1]:
# Install Requirments (Updated on 9/13/2024)
# !pip3 install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 2.6 MB/s eta 0:00:000:00:0136m0:00:01


In [2]:
import pandas as pd
import numpy as np
from yfinance import Ticker
from pykalman import KalmanFilter

%load_ext autoreload
%autoreload 2

In [3]:
def download(symbol,interval,period):
    stock = Ticker(symbol)
    stock_df = stock.history(interval=interval,
                             period=period,
                             auto_adjust=False,
                             prepost=True, # include aftermarket hours
                            )
    stock_df.to_pickle(f'./data/{symbol}_{interval}_df.pkl')
    
def load(symbol,interval):
    return pd.read_pickle(f'./data/{symbol}_{interval}_df.pkl')

In [4]:
download('NVDA','1d','max')
load('NVDA','1d')

,Open,High,Low,Close,Adj Close,Volume,Dividends,Stock Splits
Date,,,,,,,,
1999-01-22 00:00:00-05:00,0.043750,0.048828,0.038802,0.041016,0.037618,2714688000,0.00,0.0
1999-01-25 00:00:00-05:00,0.044271,0.045833,0.041016,0.045313,0.041559,510480000,0.00,0.0
1999-01-26 00:00:00-05:00,0.045833,0.046745,0.041146,0.041797,0.038334,343200000,0.00,0.0
1999-01-27 00:00:00-05:00,0.041927,0.042969,0.039583,0.041667,0.038215,244368000,0.00,0.0
1999-01-28 00:00:00-05:00,0.041667,0.041927,0.041276,0.041536,0.038095,227520000,0.00,0.0
...,...,...,...,...,...,...,...,...
2024-09-09 00:00:00-04:00,104.879997,106.550003,103.690002,106.470001,106.460892,273912000,0.00,0.0
2024-09-10 00:00:00-04:00,107.809998,109.400002,104.949997,108.099998,108.090752,268283700,0.00,0.0
2024-09-11 00:00:00-04:00,109.389999,117.190002,107.419998,116.910004,116.900002,441422400,0.00,0.0
